In [1]:
import psycopg2
import select
import json
import pandas as pd
from datetime import datetime

# Connect to the PostgreSQL databases
conn = psycopg2.connect(dbname="test_db", user="postgres", password="root", host="127.0.0.1")
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)

conn_mart = psycopg2.connect(dbname="hts_mart", user="postgres", password="root", host="127.0.0.1")
conn_mart.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)

cur = conn.cursor()
cur_mart = conn_mart.cursor()

# Execute the actual query to get the initial data
query = """
SELECT 
    chts.time AS event_date,
    cpi.person_investigation_id,
    cp.person_id,
    cp.birthdate,
    cp.sex,
    chts.date_of_hiv_test,
    chts.purpose AS reason_for_hiv_test,
    cpi.result AS hts_test_result,
    chts.approach AS hts_approach,
    chts.model AS hts_model,
    chts.hts_number,
    chts.hts_type, 
    EXTRACT(YEAR FROM age(chts.time, cp.birthdate)) AS age_at_visit
FROM 
    consultation.hts chts
LEFT JOIN 
    consultation.person_investigation cpi ON chts.laboratory_investigation_id = cpi.person_investigation_id
LEFT JOIN 
    client.person cp ON cpi.person_id = cp.person_id
"""

cur.execute(query)
rows = cur.fetchall()

# Define column names based on the query and add date_created
columns = [
    'event_date', 'person_investigation_id', 'person_id', 'birthdate', 'sex',
    'date_of_hiv_test', 'reason_for_hiv_test', 'hts_test_result', 'hts_approach',
    'hts_model', 'hts_number', 'hts_type', 'age_at_visit', 'date_created'
]

# Create a DataFrame with the initial data and current datetime as `date_created`
df = pd.DataFrame(rows, columns=columns[:-1])  # Exclude 'date_created'
df['date_created'] = datetime.now()

df = df[df['event_date'].notna()]


print("Initial DataFrame size:\n", df.shape)

# Insert or update the initial data into the `hts` table in `hts_mart`
for _, row in df.iterrows():
    hts_number = row['hts_number']
    
    # Check if a record with the given hts_number exists
    cur_mart.execute("SELECT 1 FROM hts WHERE hts_number = %s", (hts_number,))
    
    if cur_mart.fetchone():
        # Update the existing record
        update_query = """
        UPDATE hts
        SET
            event_date = %s,
            person_investigation_id = %s,
            person_id = %s,
            birthdate = %s,
            sex = %s,
            date_of_hiv_test = %s,
            reason_for_hiv_test = %s,
            hts_test_result = %s,
            hts_approach = %s,
            hts_model = %s,
            hts_type = %s,
            age_at_visit = %s,
            date_created = %s
        WHERE hts_number = %s
        """
        cur_mart.execute(update_query, (
            row['event_date'], row['person_investigation_id'], row['person_id'],
            row['birthdate'], row['sex'], row['date_of_hiv_test'],
            row['reason_for_hiv_test'], row['hts_test_result'], row['hts_approach'],
            row['hts_model'], row['hts_type'], row['age_at_visit'],
            row['date_created'], hts_number
        ))
        print(f"Updated record with hts_number: {hts_number}")
    else:
        # Insert a new record
        insert_query = """
        INSERT INTO hts (
            event_date, person_investigation_id, person_id, birthdate, sex,
            date_of_hiv_test, reason_for_hiv_test, hts_test_result, hts_approach,
            hts_model, hts_number, hts_type, age_at_visit, date_created
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cur_mart.execute(insert_query, (
            row['event_date'], row['person_investigation_id'], row['person_id'],
            row['birthdate'], row['sex'], row['date_of_hiv_test'],
            row['reason_for_hiv_test'], row['hts_test_result'], row['hts_approach'],
            row['hts_model'], row['hts_number'], row['hts_type'],
            row['age_at_visit'], row['date_created']
        ))
        print(f"Inserted new record with hts_number: {hts_number}")

print("Processed initial data into hts_mart.")

# Set up the listener for notifications on mobile_hts_channel
cur.execute("LISTEN mobile_hts_channel;")
print("Waiting for notifications on channel 'mobile_hts_channel'...")

while True:
    if select.select([conn], [], [], 5) == ([], [], []):
        print("Waiting for notifications...")
    else:
        conn.poll()
        while conn.notifies:
            notify = conn.notifies.pop(0)
            print("Got NOTIFY:", notify.payload)
            
            # Handle the notification based on the table name
            try:
                payload = json.loads(notify.payload)
                table_name = payload['table']
                row_data = payload['data']
                
                if table_name == 'hts':
                    # Create the new row data with date_created
                    new_row = {
                        'event_date': row_data.get('time'),
                        'person_investigation_id': row_data.get('laboratory_investigation_id'),
                        'person_id': row_data.get('person_id'),
                        'birthdate': row_data.get('birthdate'),
                        'sex': row_data.get('sex'),
                        'date_of_hiv_test': row_data.get('date_of_hiv_test'),
                        'reason_for_hiv_test': row_data.get('purpose'),
                        'hts_test_result': row_data.get('result'),
                        'hts_approach': row_data.get('approach'),
                        'hts_model': row_data.get('model'),
                        'hts_number': row_data.get('hts_number'),
                        'hts_type': row_data.get('hts_type'),
                        'age_at_visit': row_data.get('age_at_visit'),
                        'date_created': datetime.now()
                    }

                    # Check if a record with the given hts_number exists
                    hts_number = new_row['hts_number']
                    cur_mart.execute("SELECT 1 FROM hts WHERE hts_number = %s", (hts_number,))
                    
                    if cur_mart.fetchone():
                        # Update the existing record
                        update_query = """
                        UPDATE hts
                        SET
                            event_date = %s,
                            person_investigation_id = %s,
                            person_id = %s,
                            birthdate = %s,
                            sex = %s,
                            date_of_hiv_test = %s,
                            reason_for_hiv_test = %s,
                            hts_test_result = %s,
                            hts_approach = %s,
                            hts_model = %s,
                            hts_type = %s,
                            age_at_visit = %s,
                            date_created = %s
                        WHERE hts_number = %s
                        """
                        cur_mart.execute(update_query, (
                            new_row['event_date'], new_row['person_investigation_id'], new_row['person_id'],
                            new_row['birthdate'], new_row['sex'], new_row['date_of_hiv_test'],
                            new_row['reason_for_hiv_test'], new_row['hts_test_result'], new_row['hts_approach'],
                            new_row['hts_model'], new_row['hts_type'], new_row['age_at_visit'],
                            new_row['date_created'], hts_number
                        ))
                        print(f"Updated record with hts_number: {hts_number}")
                    else:
                        # Insert a new record
                        insert_query = """
                        INSERT INTO hts (
                            event_date, person_investigation_id, person_id, birthdate, sex,
                            date_of_hiv_test, reason_for_hiv_test, hts_test_result, hts_approach,
                            hts_model, hts_number, hts_type, age_at_visit, date_created
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        """
                        cur_mart.execute(insert_query, (
                            new_row['event_date'], new_row['person_investigation_id'], new_row['person_id'],
                            new_row['birthdate'], new_row['sex'], new_row['date_of_hiv_test'],
                            new_row['reason_for_hiv_test'], new_row['hts_test_result'], new_row['hts_approach'],
                            new_row['hts_model'], new_row['hts_number'], new_row['hts_type'],
                            new_row['age_at_visit'], new_row['date_created']
                        ))
                        print(f"Inserted new record with hts_number: {hts_number}")
                else:
                    print(f"Ignoring change from {table_name}")
            except json.JSONDecodeError:
                print("Error decoding JSON:", notify.payload)
            except Exception as e:
                print("An error occurred:", str(e))

Initial DataFrame size:
 (12830, 14)
Inserted new record with hts_number: 09-0A-27-2022-V-03300
Inserted new record with hts_number: 09-0A-27-2022-V-03296
Inserted new record with hts_number: 09-0A-27-2022-V-01746
Inserted new record with hts_number: 09-0A-27-2022-V-02854
Inserted new record with hts_number: 09-0A-27-2022-V-04487
Inserted new record with hts_number: 09-0A-27-2022-V-04489
Inserted new record with hts_number: 09-0A-27-2021-V-01143
Inserted new record with hts_number: 09-0A-27-2022-V-02130
Inserted new record with hts_number: 09-0A-27-2022-V-04421
Inserted new record with hts_number: 09-0A-27-2022-V-03911
Inserted new record with hts_number: 09-0A-27-2022-V-04181
Inserted new record with hts_number: 09-0A-27-2020-V-09497
Inserted new record with hts_number: 09-0A-27-2021-V-05263
Inserted new record with hts_number: 09-0A-27-2022-V-03925
Inserted new record with hts_number: 09-0A-27-2021-V-01203
Inserted new record with hts_number: 09-0A-27-2022-V-04138
Inserted new record

KeyboardInterrupt: 